<a href="https://colab.research.google.com/github/sadikul1500/Deciosion_tree/blob/main/id3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import pandas as pd
import numpy as np
import random
from random import shuffle
from pprint import pprint

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving parkinsons.csv to parkinsons (1).csv


In [ ]:
def split_dataset(data, start, end):
  #size = round((1/size)*len(data))

  test_indices = [i for i in range(start, end)]
  #test_indices = random.sample(indices, size)

  test = data.loc[test_indices]
  train = data.drop(test_indices)

  return train, test

In [ ]:
#print([i for i in range(start, end)])

In [ ]:
def isPure(data):
  labels = data[:, -1]
  output_classes = np.unique(labels)
  
  return len(output_classes) == 1

In [ ]:
def classify_data(data):
  labels = data[:, -1]
  output_classes, count_output_classes = np.unique(labels, return_counts=True)

  return output_classes[count_output_classes.argmax()]  

In [ ]:
def split_data(data, column, value):
  rows = data[:, column]
  feature_type = FEATURE_TYPES[column]

  if feature_type == "continuous":
    return data[rows < value], data[rows >= value]
  else:
    return data[rows == value], data[rows != value]

In [ ]:
def get_entropy(data):
  _, unique_vlaue_counts = np.unique(data[:, -1], return_counts=True)

  probabilities = unique_vlaue_counts / unique_vlaue_counts.sum()

  return -1 * sum(probabilities*np.log2(probabilities))

In [ ]:
def get_overall_entropy(data_below, data_upper):
  total_size = len(data_below) + len(data_upper)

  return ((len(data_below) / total_size) * get_entropy(data_below)) + ((len(data_upper) / total_size) * get_entropy(data_upper))

In [ ]:
def determine_best_split(data, potential_splits):
  overall_entropy = 1e9
  for  key in potential_splits.keys():
    for value in potential_splits[key]:
      data_below, data_upper = split_data(data, key, value)
      current_entropy = get_overall_entropy(data_below, data_upper)

      if current_entropy < overall_entropy:
        overall_entropy = current_entropy
        best_split_column = key
        best_split_value = value
  
  return best_split_column, best_split_value

In [ ]:
def get_potential_split(data):
  potential_splits = {}
  _, n_column = data.shape

  for col in range(n_column-1):
    #print(data[:, col])    
    unique_values = np.unique(data[:, col])
    
    if FEATURE_TYPES[col] == "continuous":
      potential_splits[col] = []
      for index in range(1, len(unique_values)):
        potential_splits[col] += [(unique_values[index] + unique_values[index-1]) / 2]
  
    elif len(unique_values) > 1:
      potential_splits[col] = unique_values
  
  return potential_splits


In [ ]:
def decision_tree_algorithm(data, column_heading, counter=0, min_point=2, max_depth=6):
  
  if isPure(data) or len(data) < 2 or counter > max_depth:
    return classify_data(data)

  else:
    #counter += 1
    potential_splits = get_potential_split(data)
    #print(potential_splits)
    if potential_splits:
      split_column, split_value = determine_best_split(data, potential_splits)
      data_below, data_upper = split_data(data, split_column, split_value)
    else:
      return classify_data(data)
      
    if FEATURE_TYPES[split_column] == "continuous":
      question = "{} < {}".format(column_heading[split_column], split_value)
    else:
      question = "{} = {}".format(column_heading[split_column], split_value)
    subtree = {question:[]}

    yes_answer = decision_tree_algorithm(data_below, column_heading, counter+1, min_point, max_depth)
    no_answer = decision_tree_algorithm(data_upper, column_heading, counter+1, min_point, max_depth)

    subtree[question].append(yes_answer)
    subtree[question].append(no_answer)

    return subtree

In [ ]:
#print(get_overall_entropy(split_data(train.values, 3, .8)))
print(np.unique(['a','b','a','d','e','c','e',5, 7]))

['5' '7' 'a' 'b' 'c' 'd' 'e']


In [ ]:
#print(test.iloc[2][-1])

In [ ]:
def classify_test_data(data, tree):
  question = list(tree.keys())[0]
  feature, compare, value = question.split()

  if compare == "<":
    if data[feature] < float(value):
      answer = tree[question][0]
    else:
      answer = tree[question][1]
  
  else:
    if str(data[feature]) == value:
      answer = tree[question][0]
    else:
      answer = tree[question][1]

  if not isinstance(answer, dict):
    return answer
  else:
    return classify_test_data(data, answer)  

#data = test.iloc[0]

In [ ]:
def get_accuracy(test, tree):
  count = 0
  total = len(test)

  print('******************** testing ************************\n')
  for i in range(total):
    row = test.iloc[i]
    answer = classify_test_data(row, tree)

    if answer == row[-1]:
      count += 1
      print(row[-1], answer)
    else: print(row[-1], answer, " WRONG_AMSWER_DETECTED")
  return count / total 

In [ ]:
FEATURE_TYPES = []

In [ ]:
def get_type_of_feature(df):
  feature_types = []
  unique_value_threshold = 15

  for col  in df.columns:
    val = df[col].unique()[0]

    if isinstance(val, str) or len(df[col].unique()) < unique_value_threshold:
      feature_types.append("categorical")
    else:
      feature_types.append("continuous")
  return feature_types

In [ ]:
for f in uploaded.keys():
  csv_file = pd.read_csv(f)
  csv_file = csv_file.rename(columns={list(csv_file.columns)[-1] : "label"})
  column_heading = csv_file.columns
  print(csv_file.info())
  
  global FEATURE_TYPES
  FEATURE_TYPES = get_type_of_feature(csv_file)
  i = 0
  for column in csv_file.columns:
    print(column, FEATURE_TYPES[i])
    i += 1

  #shuffle dataset
  csv_file = csv_file.sample(frac = 1)

  numberOfFold = int(input("number of fold: "))
  percent = int(len(csv_file) / numberOfFold)

  accuracy = []

  for t in range(numberOfFold):
    start = t * percent
    end = start + percent
    #print(start, end)
    train, test = split_dataset(csv_file, start, end )

    #potential_splits = get_potential_split(train.values)
    tree = decision_tree_algorithm(train.values, column_heading, max_depth=10)
    #print(tree)
    print('\n############ Tree ##########\n')
    pprint(tree)
    print('\n')
    accuracy += [get_accuracy(test, tree)]
    print("***********accuracy: ", accuracy[-1])


    #print(test)
    #print(isPure(train.values))
    #csv_file = csv_file.drop()
  print("**********mean accuracy: ", sum(accuracy)/ numberOfFold)

In [ ]:
print(len(csv_file))

195


In [ ]:
'''
  heading = list(csv_file.columns)

  data = csv_file.values.tolist()

  shuffle(data)
  shuffledData = list.copy(data)

  numberOfFold = int(input("number of fold: "))
  percent = int(len(shuffledData) / numberOfFold)

  accuracy = []

  for i in range(numberOfFold):
    start = i * percent
    end = start + percent
    
    test = np.array(shuffledData[start : end])
    train = np.array(shuffledData[: start] + shuffledData[end-1:])

    #test_df = np.asarray(test)
    #train_df = np.asarray(train)

    #print(type(test))
    #print(start, end)
    print(train)
    #tree = build_tr#ee(train, heading)
'''

'\n  heading = list(csv_file.columns)\n\n  data = csv_file.values.tolist()\n\n  shuffle(data)\n  shuffledData = list.copy(data)\n\n  numberOfFold = int(input("number of fold: "))\n  percent = int(len(shuffledData) / numberOfFold)\n\n  accuracy = []\n\n  for i in range(numberOfFold):\n    start = i * percent\n    end = start + percent\n    \n    test = np.array(shuffledData[start : end])\n    train = np.array(shuffledData[: start] + shuffledData[end-1:])\n\n    #test_df = np.asarray(test)\n    #train_df = np.asarray(train)\n\n    #print(type(test))\n    #print(start, end)\n    print(train)\n    #tree = build_tr#ee(train, heading)\n'

In [ ]:
'''
def information_gain(left, right, parent_entropy):
  probability = len(right) * 1.0 / (len(right) + len(left))
  return parent_entropy - probability * entropy(left) - (1-probability) * entropy(right)
'''

'\ndef information_gain(left, right, parent_entropy):\n  probability = len(right) * 1.0 / (len(right) + len(left))\n  return parent_entropy - probability * entropy(left) - (1-probability) * entropy(right)\n'

In [ ]:
'''
for f in uploaded.keys():
  df = pd.read_csv(f)
  df["label"] = df.Survived
  df = df.drop(["PassengerId", "Name", "Survived", "Ticket", "Cabin"], axis=1)
 
  df = df.fillna({"Age":df.Age.median(), "Embarked": df.Embarked.mode()[0]})
  column_heading = df.columns
  #print(df.head())
  print(df.info()) 

  global FEATURE_TYPES
  FEATURE_TYPES = get_type_of_feature(df)
  i = 0
  for column in df.columns:
    print(column, FEATURE_TYPES[i])
    i += 1
  train, test = split_dataset(df, 5)
  tree = decision_tree_algorithm(train.values, column_heading, max_depth=10)
  #print(tree)
  #print()
  pprint(tree, width=50)
  print('\n')
  print(get_accuracy(test, tree))
'''

'\nfor f in uploaded.keys():\n  df = pd.read_csv(f)\n  df["label"] = df.Survived\n  df = df.drop(["PassengerId", "Name", "Survived", "Ticket", "Cabin"], axis=1)\n \n  df = df.fillna({"Age":df.Age.median(), "Embarked": df.Embarked.mode()[0]})\n  column_heading = df.columns\n  #print(df.head())\n  print(df.info()) \n\n  global FEATURE_TYPES\n  FEATURE_TYPES = get_type_of_feature(df)\n  i = 0\n  for column in df.columns:\n    print(column, FEATURE_TYPES[i])\n    i += 1\n  train, test = split_dataset(df, 5)\n  tree = decision_tree_algorithm(train.values, column_heading, max_depth=10)\n  #print(tree)\n  #print()\n  pprint(tree, width=50)\n  print(\'\n\')\n  print(get_accuracy(test, tree))\n'